In [1]:
import numpy as np
from scipy.linalg import eigh
import matplotlib.pyplot as plt
import pandas as pd
import pickle

np.set_printoptions(precision=6)

# initialize variables
N = 8  # Number of spins
Q = pow(2, N)  # Dimension of the Hilbert space for N spins

def D2B(num, digit):
    string = f'{num:0{digit}b}'  # Convert to binary string with leading zeros
    result = np.array([int(ele) for ele in string], int)  # Convert each bit to an integer array
    return result

def B2D(array):
    res = 0
    for ele in array:
        res = (res << 1) | ele  # Left shift and add the current bit
    return res

def spin_reflection(array):
    return 1 - array  # Flip each spin (1 -> 0, 0 -> 1)

def calculate_spin_parity(eigenvector):
    parity_sum = 0  # Initialize the parity sum for the current eigenvector
    for basis_idx in range(Q):  # Loop over all basis states
        basis = D2B(basis_idx, N)  # Convert basis index to binary representation
        reflected_basis = spin_reflection(basis)  # Apply spin reflection to the basis state
        reflected_idx = B2D(reflected_basis)  # Convert the reflected basis back to a decimal index

        # Calculate the parity by comparing original and reflected amplitudes
        parity_sum += eigenvector[basis_idx] * eigenvector[reflected_idx]

    return round(parity_sum, 5)  # Round to 5 digits

# Initialize parameters
dB = 0.1  # Step size for B
B_not = 10  # Example maximum value for B
B_end = 0.1
B_value_list = np.arange(B_end, B_not, dB)  # Generate list of B values
num_matching_states = 5  # Number of matching excited states to find
gamma = 1  # Set gamma to be 1
t_f_list = []  # List to store t_f values for each random J_ij matrix
J_ij_t_f_dict = {}  # Dictionary to store J_ij matrices and corresponding t_f values
number_of_iteration = 10000
best_t_f = 0  # Initialize best t_f as 0
best_J_ij = None  # Initialize best J_ij matrix

# Precompute the transverse field term (X term)
HB_matrix = np.zeros((Q, Q))  # Initialize the transverse field term (X term)
for state in range(Q):  # Loop over all possible basis states
    basis = D2B(state, N)  # Convert the current state index to a binary representation
    for i in range(N):  # Loop over each spin in the state
        flipped_basis = basis.copy()  # Copy the current basis state
        flipped_basis[i] = 1 - flipped_basis[i]  # Flip the i-th spin
        new_state = B2D(flipped_basis)  # Convert the flipped basis back to a decimal index
        HB_matrix[state, new_state] += 1  # Add the contribution to the X term

# Loop over 100 different random J_ij matrices
for iteration in range(number_of_iteration):
    print(f"Iteration {iteration + 1}")

    # Generate a random J_ij matrix for spin glass model
    J_matrix = np.triu(np.random.normal(0, 1, (N, N)), 1)

    # Hamiltonian construction
    H_a = np.zeros(Q)  # Initialize the interaction term (ZZ term), which is diagonal

    # Construct the Hamiltonian
    for state in range(Q):  # Loop over all possible basis states
        basis = D2B(state, N)  # Convert the current state index to a binary representation

        # Interaction term (ZZ term)
        for i in range(N):  # Loop over each spin
            for j in range(i + 1, N):  # Loop over pairs of spins (i, j)
                J_ij = J_matrix[i][j]  # Interaction strength between spin i and j
                # Add the contribution to the ZZ term
                H_a[state] += J_ij * (1 if basis[i] == basis[j] else -1)

    # Convert H_a to a diagonal matrix
    HA_matrix = np.diag(H_a)

    # Calculate energy gaps for different B values
    delta_e_lists = [[] for _ in range(num_matching_states)]  # List to store energy gaps for matching states

    for B_value in B_value_list:
        H_matrix = HA_matrix + B_value * HB_matrix  # Update Hamiltonian with current B value
        vals, vecs = eigh(H_matrix)  # Diagonalize the Hamiltonian

        # Get ground state parity
        gs_parity = calculate_spin_parity(vecs[:, 0])
        # Find the first few excited states with the same parity as the ground state
        matching_indices = []
        for i in range(1, Q):
            if calculate_spin_parity(vecs[:, i]) == gs_parity:
                matching_indices.append(i)
            if len(matching_indices) >= num_matching_states:
                break

        # Calculate energy gaps for the matching states
        for idx, matching_index in enumerate(matching_indices):
            delta_energy = vals[matching_index] - vals[0]
            delta_e_lists[idx].append(delta_energy)

    # Calculate total ramp time t_f for locally adiabatic evolution for the first matching state
    if len(delta_e_lists[0]) == len(B_value_list):  # Ensure matching length for integration
        delta_e_array = np.array(delta_e_lists[0])
        one_over_delta_square = 1 / np.square(delta_e_array)  # Calculate 1 / delta^2 for the first matching state
        t_f = gamma * np.trapz(one_over_delta_square, x=B_value_list)  # Use trapezoidal integration to calculate t_f
        t_f_list.append(t_f)
        print('t_f is ', t_f)

        # Store J_ij matrix and t_f value in the dictionary
        J_ij_t_f_dict[iteration] = {'J_ij': J_matrix, 't_f': t_f}

        # Check if the current t_f is the best one found so far
        if t_f > best_t_f:
            best_t_f = t_f
            best_J_ij = J_matrix


Iteration 1
t_f is  1.489009795999302
Iteration 2


KeyboardInterrupt: 

In [ ]:
# # Save t_f values to a CSV file
# df_tf = pd.DataFrame({'t_f': t_f_list})
# df_tf.to_csv('t_f_values.csv', index=False)

# # Save the J_ij and t_f dictionary to a pickle file
# with open('J_ij_t_f_dict.pkl', 'wb') as f:
#     pickle.dump(J_ij_t_f_dict, f)

# # Save the best J_ij matrix to a pickle file
# with open('best_J_ij.pkl', 'wb') as f:
#     pickle.dump({'best_J_ij': best_J_ij, 'best_t_f': best_t_f}, f)

# # Read the J_ij and t_f dictionary from the pickle file
# with open('J_ij_t_f_dict.pkl', 'rb') as f:
#     loaded_J_ij_t_f_dict = pickle.load(f)

# # Print the loaded data to verify
# print(loaded_J_ij_t_f_dict)

# # Read the best J_ij matrix from the pickle file
# with open('best_J_ij.pkl', 'rb') as f:
#     loaded_best_J_ij = pickle.load(f)

# # Print the best J_ij matrix and corresponding t_f value
# print('Best J_ij matrix:', loaded_best_J_ij['best_J_ij'])
# print('Best t_f value:', loaded_best_J_ij['best_t_f'])